## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
#Just want to make sure all the columns are visible
#==================================================
pd.options.display.max_columns=None

In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
#======================================================
df = application_df.drop(columns=['EIN', 'NAME', 'SPECIAL_CONSIDERATIONS'], axis=1)
df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,142590,1
...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,5000,1


In [4]:
# Determine the number of unique values in each column.
#======================================================
df.nunique()

APPLICATION_TYPE      17
AFFILIATION            6
CLASSIFICATION        71
USE_CASE               5
ORGANIZATION           4
STATUS                 2
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

In [5]:
# Look at APPLICATION_TYPE value counts for binning
#===================================================
app_type_cnt = df['APPLICATION_TYPE'].value_counts()
app_type_cnt

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [6]:
#Choose a cutoff value and create a list of application types to be replaced
#use the variable name `application_types_to_replace`
#===================================================================================
application_types_to_replace = list(app_type_cnt[app_type_cnt <500].index)

In [7]:
#Replace in dataframe
#===============================================================================================
for app in application_types_to_replace:
    df['APPLICATION_TYPE'] = df['APPLICATION_TYPE'].replace(app,"Other")

In [8]:
#Check to make sure binning was successful
#==================================================
df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [9]:
#Look at CLASSIFICATION value counts for binning
#=================================================
class_val_cnt = df['CLASSIFICATION'].value_counts()
class_val_cnt

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4200        1
C1236        1
C1570        1
C1248        1
C1283        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [10]:
#You may find it helpful to look at CLASSIFICATION value counts >1
#===================================================================
class_val_cnt.loc[lambda x : x>1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C2300       32
C7200       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1278       10
C1238       10
C1235        9
C1237        9
C7210        7
C4100        6
C1720        6
C2400        6
C1600        5
C1257        5
C0           3
C2710        3
C1260        3
C1256        2
C1267        2
C1234        2
C1246        2
C3200        2
Name: CLASSIFICATION, dtype: int64

In [11]:
#Choose a cutoff value and create a list of classifications to be replaced
#use the variable name `classifications_to_replace`
#=========================================================================
classifications_to_replace = list(class_val_cnt[class_val_cnt <500].index)

In [12]:
#Replace in dataframe
#============================================================================================
for cls in classifications_to_replace:
    df['CLASSIFICATION'] = df['CLASSIFICATION'].replace(cls,"Other")
    

In [13]:
#Check to make sure binning was successful
#================================================
df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [14]:
#Convert categorical data to numeric with `pd.get_dummies`
#============================================================
app_data = pd.get_dummies(df)
app_data

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C2000,CLASSIFICATION_C2100,CLASSIFICATION_C3000,CLASSIFICATION_C7000,CLASSIFICATION_Other,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,5000,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0
1,1,108590,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0
2,1,5000,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0
3,1,6692,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0
4,1,142590,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0
34295,1,5000,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0
34296,1,5000,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0
34297,1,5000,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0


In [15]:
#Split our preprocessed data into our features and target arrays
#=================================================================
X = app_data.drop(['IS_SUCCESSFUL'],1).values
y = app_data['IS_SUCCESSFUL'].values

In [16]:
#Split the preprocessed data into a training and testing dataset
#=================================================================
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=41)

In [17]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [36]:
#Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#=========================================================================================================
number_of_input_features = len(X_train[0])
hidden_node1 = 100
hidden_node2 = 80
hidden_node3 = 65
#hidden_node4 = 45
#hidden_node5 = 30

nn = tf.keras.models.Sequential()

In [37]:
#First hidden layer
#=======================================================================================================
nn.add(tf.keras.layers.Dense(units=hidden_node1, input_dim=number_of_input_features, activation="relu"))

In [38]:
#Second hidden layer
#=====================================================================
nn.add(tf.keras.layers.Dense(units=hidden_node2, activation="relu"))

In [39]:
#Third hidden layer
#=====================================================================
nn.add(tf.keras.layers.Dense(units=hidden_node3, activation="tanh"))

In [ ]:
#Fourth hidden layer
#=====================================================================
#nn.add(tf.keras.layers.Dense(units=hidden_node4, activation="linear"))

In [ ]:
#Fifth hidden layer
#=====================================================================
#nn.add(tf.keras.layers.Dense(units=hidden_node5, activation="relu"))

In [40]:
#Output layer
#============================================================
nn.add(tf.keras.layers.Dense(units=1, activation="relu"))

In [41]:
#Check the structure of the model
#================================
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 100)               4300      
                                                                 
 dense_9 (Dense)             (None, 80)                8080      
                                                                 
 dense_10 (Dense)            (None, 65)                5265      
                                                                 
 dense_11 (Dense)            (None, 1)                 66        
                                                                 
Total params: 17,711
Trainable params: 17,711
Non-trainable params: 0
_________________________________________________________________


In [42]:
#Compile the model
#====================================================================================
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [43]:
#Train the model
#=====================================================
fit_model = nn.fit(X_train_scaled, y_train, epochs=73)

Epoch 1/73
804/804 [==============================] - 1s 673us/step - loss: 8.2390 - accuracy: 0.4659
Epoch 2/73
804/804 [==============================] - 1s 657us/step - loss: 8.2390 - accuracy: 0.4659
Epoch 3/73
804/804 [==============================] - 1s 653us/step - loss: 8.2390 - accuracy: 0.4659
Epoch 4/73
804/804 [==============================] - 1s 650us/step - loss: 8.2390 - accuracy: 0.4659
Epoch 5/73
804/804 [==============================] - 1s 656us/step - loss: 8.2390 - accuracy: 0.4659
Epoch 6/73
804/804 [==============================] - 1s 644us/step - loss: 8.2390 - accuracy: 0.4659
Epoch 7/73
804/804 [==============================] - 1s 657us/step - loss: 8.2390 - accuracy: 0.4659
Epoch 8/73
804/804 [==============================] - 1s 653us/step - loss: 8.2390 - accuracy: 0.4659
Epoch 9/73
804/804 [==============================] - 1s 674us/step - loss: 8.2390 - accuracy: 0.4659
Epoch 10/73
804/804 [==============================] - 1s 694us/step - loss: 8.239

In [44]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 8.1325 - accuracy: 0.4728 - 154ms/epoch - 574us/step
Loss: 8.1325044631958, Accuracy: 0.47276967763900757


In [ ]:
#Export our model to HDF5 file
#===============================
#nn.save("AlphaSoupCharity.h5")